# Lambda Mart vs KNN vs SVD

Ниже посмотрим на качество ранжирования с помощью LambdaMart на задаче MQ, на различные метрики ранжирования, а также на LambdaMart + KNN против SVD, на задаче MovieLens

Все данные можно найти здесь https://onedrive.live.com/?authkey=%21ACnoZZSZVfHPJd0&id=8FEADC23D838BDA8%21107&cid=8FEADC23D838BDA8

In [1]:
from surprise import Dataset
from surprise import Reader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold

from knn_inherited import KNNMeans, KNNBasic

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree


from svd import SVD

from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings("ignore")

import LETOR


seed = 0xAB0BA
np.random.seed(seed)

# MQ2007

Рассмотрим сначала датасет 2007 года, посчитаем на изначальном датасете все метрики и посмотрим сколько мы сможем выбить нашим LambdaMart-ом

In [2]:
df = pd.read_csv('train_2007.txt', sep=" ", header=None)

Переделаем под удобный вид, обзовем колонки приятными именами

In [3]:
prob = df[56]
doc_id = df[50]
df = df.iloc[:,:48]
df['doc_id'] = doc_id
for i in range(1, 48):
    df[i] = df[i].str.split(':').str[1]
    df[i] = df[i].astype(float)
    
df[1] = df[1].astype(int)
df['prev_res'] = prob

names = ['relevance', 'query_id']
for i in range(2, 48):
    names.append('feature' + str(i))
    
names.append('doc_id')
names.append('prev_res')

df.columns = names

In [4]:
df.head()

,relevance,query_id,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature40,feature41,feature42,feature43,feature44,feature45,feature46,feature47,doc_id,prev_res
0,0,10,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.0,GX000-00-0000000,0.024691
1,1,10,0.031310,0.666667,0.50,0.166667,0.033206,0.0,0.0,0.0,...,0.823908,0.750092,0.385426,0.923077,0.086207,0.333333,0.448276,0.0,GX000-24-12369390,0.416367
2,1,10,0.078682,0.166667,0.50,0.333333,0.080022,0.0,0.0,0.0,...,0.868557,0.641385,0.010462,0.076923,0.074713,0.833333,0.678161,0.0,GX000-62-7863450,0.568950
3,1,10,0.019058,1.000000,1.00,0.500000,0.022591,0.0,0.0,0.0,...,1.000000,0.863460,0.016642,0.153846,0.040230,0.833333,0.896552,0.0,GX016-48-5543459,0.775913
4,0,10,0.039477,0.000000,0.75,0.166667,0.040555,0.0,0.0,0.0,...,0.769845,0.646567,0.073711,0.076923,0.034483,0.333333,0.218391,0.0,GX037-87-3082362,0.334800


Разделим выборку на тренировочную и тестовую в соотношение 4 к 1. Будем делить по запросам, чтобы ничего не сломать.

In [5]:
n = len(df['query_id'].unique())
train_idx = np.random.choice(df['query_id'].unique().astype(int), int(n*0.8), replace=False)

df_train = df.loc[df['query_id'].isin(train_idx)]
df_test = df.loc[~(df['query_id'].isin(train_idx))]

Рассчитаем метрики MRR, MAP и усредненный nDCG по всем запросам. Примем за ответ признак prob от авторов

In [6]:
print('Train metrics')
print(' MRR = ', LETOR.MRR(df_train), '\n',
      'MAP@10 = ', LETOR.MAP(df_train, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(df_train, 10))

print('---------')
print('Test metrics')
print(' MRR = ', LETOR.MRR(df_test), '\n',
      'MAP@10 = ', LETOR.MAP(df_test, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(df_test, 10))

Train metrics
 MRR =  0.5801735839060365 
 MAP@10 =  0.31203576797672744 
 mean nDCG@10 =  0.5043703263095799
---------
Test metrics
 MRR =  0.6080969858243465 
 MAP@10 =  0.30883267195767194 
 mean nDCG@10 =  0.5057646751681216


А теперь обучим Лямбду март и посмотрим на качество

In [7]:
lambda_mart = LETOR.LambdaMart(n_trees = 5)
lambda_mart.fit(df_train, verbose = False)

In [8]:
X_pred = lambda_mart.predict(df_test)

In [9]:
print('Train metrics')
print(' MRR = ', LETOR.MRR(lambda_mart.X), '\n',
      'MAP@10 = ', LETOR.MAP(lambda_mart.X, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(lambda_mart.X, 10))

print('---------')
print('Test metrics')
print(' MRR = ', LETOR.MRR(X_pred), '\n',
      'MAP@10 = ', LETOR.MAP(X_pred, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(X_pred, 10))

Train metrics
 MRR =  0.548924337761267 
 MAP@10 =  0.27663362228860383 
 mean nDCG@10 =  0.47261973821990033
---------
Test metrics
 MRR =  0.5482820855977371 
 MAP@10 =  0.2786142234671647 
 mean nDCG@10 =  0.4737965854369945


Наши метрики немного хуже, но не сильно! И кажется без всяких трюков и подбора гиперпараметров это уже неплохой результат

# MQ2008

Посмотрим все то же самое на датасете 2008 года

In [10]:
df = pd.read_csv('train_2008.txt', sep=" ", header=None)

prob = df[56]
doc_id = df[50]
df = df.iloc[:,:48]
df['doc_id'] = doc_id
for i in range(1, 48):
    df[i] = df[i].str.split(':').str[1]
    df[i] = df[i].astype(float)
    
df[1] = df[1].astype(int)
df['prev_res'] = prob

names = ['relevance', 'query_id']
for i in range(2, 48):
    names.append('feature' + str(i))
    
names.append('doc_id')
names.append('prev_res')

df.columns = names

n = len(df['query_id'].unique())
train_idx = np.random.choice(df['query_id'].unique().astype(int), int(n*0.8), replace=False)

df_train = df.loc[df['query_id'].isin(train_idx)]
df_test = df.loc[~(df['query_id'].isin(train_idx))]

In [11]:
print('Train metrics')
print(' MRR = ', LETOR.MRR(df_train), '\n',
      'MAP@10 = ', LETOR.MAP(df_train, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(df_train, 10))

print('---------')
print('Test metrics')
print(' MRR = ', LETOR.MRR(df_test), '\n',
      'MAP@10 = ', LETOR.MAP(df_test, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(df_test, 10))

Train metrics
 MRR =  0.568728429865887 
 MAP@10 =  0.22934807256235828 
 mean nDCG@10 =  0.45867450651825253
---------
Test metrics
 MRR =  0.505518341307815 
 MAP@10 =  0.21933603353622141 
 mean nDCG@10 =  0.4353952936603141


In [12]:
lambda_mart = LETOR.LambdaMart(n_trees = 5)
lambda_mart.fit(df_train, verbose = False)
X_pred = lambda_mart.predict(df_test)

In [13]:
print('Train metrics')
print(' MRR = ', LETOR.MRR(lambda_mart.X), '\n',
      'MAP@10 = ', LETOR.MAP(lambda_mart.X, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(lambda_mart.X, 10))

print('---------')
print('Test metrics')
print(' MRR = ', LETOR.MRR(X_pred), '\n',
      'MAP@10 = ', LETOR.MAP(X_pred, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(X_pred, 10))

Train metrics
 MRR =  0.539969078123445 
 MAP@10 =  0.21360552886934914 
 mean nDCG@10 =  0.4263591140635728
---------
Test metrics
 MRR =  0.4758685947695236 
 MAP@10 =  0.1955658193101802 
 mean nDCG@10 =  0.3982563091902146


На этом датасете изначально метрики немного хуже. Соответственно и на нашей модели они немного хуже. Что в том, что в этом разрыв примерно одинаковый

# MovieLens

Теперь рассмотрим датасет с фильмами. если оценка 5, тогда релевантность 2, если оценка 4 тогда релевантность 1, иначе 0.

Теперь наша задача привести датасет к тому виду, который может сьесть наш Лямбда март. Будем думать о юзерах как о запросах, мол они запрашивают какой фильм они хотели бы посмотреть, а мы должны им порекомендовать. В качестве документа выступит конкретный фильм, который будет релевантный или нет. Признаки нам нужны для запроса, соответственно это будет у юзера его среднее по всем фильмам, разброс по всем фильмам, минимальная оценка и максимальная. Конечно наши признаки должны отображать признак соотношение запрос-документ, в нашем случае пользователь-фильм. Первое что приходит в голову - это найти соседей этого пользователя, посчитать по соседям средние и использовать это. Сделаем сначала простые признаки фильмов, а затем признаки из КННа

In [14]:
data = Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [15]:
user_means = df.groupby('user').mean()['rating']
user_sd = df.groupby('user').std()['rating']
user_min = df.groupby('user').min()['rating']
user_max = df.groupby('user').max()['rating']

features = user_means.to_frame().merge(user_sd.to_frame(), on='user')
features = features.merge(user_min.to_frame(), on='user')
features = features.merge(user_max.to_frame(), on='user')
features.columns = ['feature1', 'feature2', 'feature3', 'feature4']
features.index.name = 'query_id'

In [16]:
df = df.drop(columns = ['timestamp'])
df.columns = ['query_id', 'doc_id', 'relevance']
df['relevance'][(df['relevance'] < 4)] = 0
df['relevance'][(df['relevance'] == 4)] = 1
df['relevance'][(df['relevance'] == 5)] = 2

In [17]:
df = df.merge(features, on='query_id', how='left')

In [18]:
df

,query_id,doc_id,relevance,feature1,feature2,feature3,feature4
0,196,242,0.0,3.615385,1.016065,1.0,5.0
1,186,302,0.0,3.413043,1.223867,1.0,5.0
2,22,377,0.0,3.351562,1.493239,1.0,5.0
3,244,51,0.0,3.651261,1.071406,1.0,5.0
4,166,346,0.0,3.550000,1.431782,1.0,5.0
...,...,...,...,...,...,...,...
99995,880,476,0.0,3.426630,0.982156,1.0,5.0
99996,716,204,2.0,3.888476,1.041408,1.0,5.0
99997,276,1090,0.0,3.465251,1.017140,1.0,5.0
99998,13,225,0.0,3.097484,1.416414,1.0,5.0


Датасет просто огромный для тех операций которые мы здесь делаем, на полном у меня упало ядро. Поэтому случайно выберем 200 фильмов которым повезло оказаться в нашей модели

In [19]:
df['doc_id'] = df['doc_id'].astype(int)
m = len(df['doc_id'].unique())
lucky_docs = np.random.choice(df['doc_id'].unique().astype(int), 200, replace=False)
df = df[(df['doc_id'].isin(lucky_docs))]

In [20]:
df

,query_id,doc_id,relevance,feature1,feature2,feature3,feature4
0,196,242,0.0,3.615385,1.016065,1.0,5.0
6,115,265,0.0,3.934783,1.174868,1.0,5.0
8,305,451,0.0,3.409910,1.079840,1.0,5.0
13,210,40,0.0,4.060606,0.817346,2.0,5.0
22,299,144,1.0,3.464286,0.907245,1.0,5.0
...,...,...,...,...,...,...,...
99942,363,181,2.0,3.054662,1.280210,1.0,5.0
99953,655,913,1.0,2.908029,0.732701,1.0,5.0
99958,394,380,1.0,3.859060,0.937364,1.0,5.0
99960,621,809,1.0,3.549708,1.052525,1.0,5.0


In [21]:
df['query_id'] = df['query_id'].astype(int)
n = len(df['query_id'].unique())
train_idx = np.random.choice(df['query_id'].unique().astype(int), int(n*0.8), replace=False)

df_train = df.loc[df['query_id'].isin(train_idx)]
df_test = df.loc[~(df['query_id'].isin(train_idx))]

In [22]:
lambda_mart = LETOR.LambdaMart(n_trees = 10, max_depth = 7, max_rank=2)
lambda_mart.fit(df_train, verbose = True)


0 iteration nDCGs == 0.39290276756423886 
1 iteration nDCGs == 0.39290276756423886 
2 iteration nDCGs == 0.39290276756423886 
3 iteration nDCGs == 0.39290276756423886 
4 iteration nDCGs == 0.39290276756423886 
5 iteration nDCGs == 0.39290276756423886 
6 iteration nDCGs == 0.39290276756423886 
7 iteration nDCGs == 0.39290276756423886 
8 iteration nDCGs == 0.39290276756423886 
9 iteration nDCGs == 0.39290276756423886 


Очевидно с такими признаками мы даже не учимся, они видимо совсем неинформативные. Теперь будем брать по нескольким соседям средние и по ним предсказывать

In [23]:
knn = KNNBasic(3)
rating_matrix = df.pivot(index='query_id',
        columns='doc_id',
        values='relevance').fillna(0)

knn.fit(rating_matrix)
knn.fill_matrix()
n3 = knn.y_df
melted_n3 = n3.melt(ignore_index=False).dropna()

df = df.merge(melted_n3, on=['query_id', 'doc_id'], how='left')
df.columns = ['query_id', 'doc_id', 'relevance', 'feature1', 'feature2', 'feature3',
       'feature4', 'feature5']
df['feature5'] = df['feature5'].fillna(0)

In [24]:
df['query_id'] = df['query_id'].astype(int)
n = len(df['query_id'].unique())
train_idx = np.random.choice(df['query_id'].unique().astype(int), int(n*0.8), replace=False)

df_train = df.loc[df['query_id'].isin(train_idx)]
df_test = df.loc[~(df['query_id'].isin(train_idx))]

In [25]:
lambda_mart = LETOR.LambdaMart(n_trees = 10, max_depth = 5, max_rank=2)
lambda_mart.fit(df_train, verbose = True)

0 iteration nDCGs == 0.38687565567014154 
1 iteration nDCGs == 0.5641782969472084 
2 iteration nDCGs == 0.5639973028390454 
3 iteration nDCGs == 0.5639973028390454 
4 iteration nDCGs == 0.5640212588167981 
5 iteration nDCGs == 0.5640212588167981 
6 iteration nDCGs == 0.5640212588167981 
7 iteration nDCGs == 0.5640212588167981 
8 iteration nDCGs == 0.5640212588167981 
9 iteration nDCGs == 0.5640212588167981 


Видим что уже учимся, попробуем добавить признаков с нескольких соседей


In [26]:
knn = KNNBasic(5)
rating_matrix = df.pivot(index='query_id',
        columns='doc_id',
        values='relevance').fillna(0)

knn.fit(rating_matrix)
knn.fill_matrix()
n5 = knn.y_df
melted_n5 = n5.melt(ignore_index=False).dropna()

df = df.merge(melted_n5, on=['query_id', 'doc_id'], how='left')
df.columns = ['query_id', 'doc_id', 'relevance', 'feature1', 'feature2', 'feature3',
       'feature4', 'feature5', 'feature6']
df['feature6'] = df['feature6'].fillna(0)

In [27]:
knn = KNNBasic(7)
rating_matrix = df.pivot(index='query_id',
        columns='doc_id',
        values='relevance').fillna(0)

knn.fit(rating_matrix)
knn.fill_matrix()
n7 = knn.y_df
melted_n7 = n7.melt(ignore_index=False).dropna()

df = df.merge(melted_n7, on=['query_id', 'doc_id'], how='left')
df.columns = ['query_id', 'doc_id', 'relevance', 'feature1', 'feature2', 'feature3',
       'feature4', 'feature5', 'feature6', 'feature7']
df['feature7'] = df['feature7'].fillna(0)

In [28]:
df['query_id'] = df['query_id'].astype(int)
n = len(df['query_id'].unique())
train_idx = np.random.choice(df['query_id'].unique().astype(int), int(n*0.8), replace=False)

df_train = df.loc[df['query_id'].isin(train_idx)]
df_test = df.loc[~(df['query_id'].isin(train_idx))]

In [29]:
lambda_mart = LETOR.LambdaMart(n_trees = 10, max_depth=5, max_rank=2)
lambda_mart.fit(df_train, verbose = True)

0 iteration nDCGs == 0.39043589925081096 
1 iteration nDCGs == 0.5773145854226704 
2 iteration nDCGs == 0.5778621846973017 
3 iteration nDCGs == 0.5780527861062831 
4 iteration nDCGs == 0.5780527861062831 
5 iteration nDCGs == 0.5781006787975719 
6 iteration nDCGs == 0.5781006787975719 
7 iteration nDCGs == 0.5777412204613096 
8 iteration nDCGs == 0.5777412204613096 
9 iteration nDCGs == 0.5777412204613096 


Качество поднялось но не сильно, признаки среднего все еще слишком слабые, а информации о соседей в количестве одного признака вполне достаточно. Думаю, такой метод бы работал, если бы мы знали что-то еще о фильмах. Описание или какие то такие признаки. Могли бы искать расстояние между эмбеддингами, доставать энтити и прочее. Посмотрим все же, как оно на тесте и другие метрики.

In [30]:
X_pred = lambda_mart.predict(df_test)

In [31]:
print('Train metrics @ 10')
print(' MRR = ', LETOR.MRR(lambda_mart.X), '\n',
      'MAP@10 = ', LETOR.MAP(lambda_mart.X, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(lambda_mart.X, 10, 2))

print('---------')
print('Test metrics @ 10')
print(' MRR = ', LETOR.MRR(X_pred), '\n',
      'MAP@10 = ', LETOR.MAP(X_pred, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(X_pred, 10, 2))


Train metrics @ 10
 MRR =  0.8854054054054055 
 MAP@10 =  0.6128931071803095 
 mean nDCG@10 =  0.5438161837481544
---------
Test metrics @ 10
 MRR =  0.8797297297297297 
 MAP@10 =  0.6251744874125826 
 mean nDCG@10 =  0.5551719508534374


Тем не менее остальные метрики весьма хорошие, лучше чем на прошлом. Но кажется это специфика задачи, здесь нам как будто бы 1) проще попасть с фильмами, больше релевантных 2) мы использовали КНН, который дает нам уже почти ответ на нашу задачу, а точнее раньше давал ответ на задачу рекомендации по 5 бальной шкале, и вообще восстанавливал оценки для каждого фильма


# SVD

Здесь используется алгоритм SVD, поэтому попробуем взять целый датасет и посмотрим. Исходя из прошлой работы будем использовать алгоритм SGD

In [32]:
data = Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']

df = df.drop(columns=['timestamp'])
df['rating'][(df['rating'] < 4)] = 0
df['rating'][(df['rating'] == 4)] = 1
df['rating'][(df['rating'] == 5)] = 2

df['user'] = df['user'].astype(int)
n = len(df['user'].unique())
train_idx = np.random.choice(df['user'].unique().astype(int), int(n*0.8), replace=False)

df_train = df.loc[df['user'].isin(train_idx)]
df_test = df.loc[~(df['user'].isin(train_idx))]

svd = SVD(15)
svd.fit(df_train, algo='sgd', verbose=None)

y_pred, y_test = svd.predict(df_test)
df_test['prev_res'] = y_pred
df_test.columns = ['query_id', 'doc_id', 'relevance', 'prev_res']

0.06523771997530782


In [33]:
y_pred, y_test = svd.predict(df_train)
df_train['prev_res'] = y_pred
df_train.columns = ['query_id', 'doc_id', 'relevance', 'prev_res']

In [34]:
print('Train metrics @ 10')
print(' MRR = ', LETOR.MRR(df_train), '\n',
      'MAP@10 = ', LETOR.MAP(df_train, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(df_train, 10, 2))

print('---------')
print('Test metrics @ 10')
print(' MRR = ', LETOR.MRR(df_test), '\n',
      'MAP@10 = ', LETOR.MAP(df_test, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(df_test, 10, 2))


Train metrics @ 10
 MRR =  0.7810324254289771 
 MAP@10 =  0.4797733779630331 
 mean nDCG@10 =  0.4502959617529576
---------
Test metrics @ 10
 MRR =  0.7812673217435123 
 MAP@10 =  0.4480179726211472 
 mean nDCG@10 =  0.4225930413134915


Рассмотрим качество на топ 10 фильмах на тесте

In [35]:
print('Test metrics @ 10 KNN+LambdaMart')
print(' MRR = ', LETOR.MRR(X_pred), '\n',
      'MAP@10 = ', LETOR.MAP(X_pred, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(X_pred, 10, 2))


print('---------')
print('Test metrics @ 10 SVD')
print(' MRR = ', LETOR.MRR(df_test), '\n',
      'MAP@10 = ', LETOR.MAP(df_test, 10), '\n',
      'mean nDCG@10 = ', LETOR.mean_nDCG(df_test, 10, 2))

Test metrics @ 10 KNN+LambdaMart
 MRR =  0.8797297297297297 
 MAP@10 =  0.6251744874125826 
 mean nDCG@10 =  0.5551719508534374
---------
Test metrics @ 10 SVD
 MRR =  0.7812673217435123 
 MAP@10 =  0.4480179726211472 
 mean nDCG@10 =  0.4225930413134915


по всем метрикам на топ10 фильмах выигрывает кнн и лямбда март. Посмотрим на топ 5 фильмов

In [36]:
print('Test metrics @ 5 KNN + LambdaMart')
print(' MRR = ', LETOR.MRR(X_pred), '\n',
      'MAP@5 = ', LETOR.MAP(X_pred, 5), '\n',
      'mean nDCG@5 = ', LETOR.mean_nDCG(X_pred, 5, 2))


print('---------')
print('Test metrics @ 5 SVD')
print(' MRR = ', LETOR.MRR(df_test), '\n',
      'MAP@5 = ', LETOR.MAP(df_test, 5), '\n',
      'mean nDCG@5 = ', LETOR.mean_nDCG(df_test, 5, 2))

Test metrics @ 5 KNN + LambdaMart
 MRR =  0.8797297297297297 
 MAP@5 =  0.6807237237237238 
 mean nDCG@5 =  0.6176424813557154
---------
Test metrics @ 5 SVD
 MRR =  0.7812673217435123 
 MAP@5 =  0.5022222222222222 
 mean nDCG@5 =  0.4343452176046694


Видим, что КНН с лямбда мартом опять оказался лучше. При этом видим, что nDCG и MAP вырос у алгоритма лямбда март сильнее. Теперь посмотрим на топ-3

In [37]:
print('Test metrics @ 3 KNN + LambdaMart')
print(' MRR = ', LETOR.MRR(X_pred), '\n',
      'MAP@3 = ', LETOR.MAP(X_pred, 3), '\n',
      'mean nDCG@3 = ', LETOR.mean_nDCG(X_pred, 3, 2))

print('---------')

print('Test metrics @ 3 SVD')
print(' MRR = ', LETOR.MRR(df_test), '\n',
      'MAP@3 = ', LETOR.MAP(df_test, 3), '\n',
      'mean nDCG@3 = ', LETOR.mean_nDCG(df_test, 3, 2))



Test metrics @ 3 KNN + LambdaMart
 MRR =  0.8797297297297297 
 MAP@3 =  0.7385885885885886 
 mean nDCG@3 =  0.6704034467206315
---------
Test metrics @ 3 SVD
 MRR =  0.7812673217435123 
 MAP@3 =  0.5511463844797178 
 mean nDCG@3 =  0.4419614882221018


опять же метрики лучше у первого алгоритма. Видим, что на 3 обьектах nDCG становится еще лучше и MAP тоже, однако nDCG не стал сильно лучше у SVD.

Видим, что на lambdamart с кнн в качестве признаков качество выше. Этому есть 3 обьяснения. 1) в предыдущих работах на тему этого датасета получалось то же самое. Скорее всего это связано со спецификой данных, возможно с небольшим количеством данных, что нам проще искать соседей. 2) в алгоритме с лямбда мартом мы обучаем алгоритм непосредственно на nDCG и смотрим что будет при перестановках в ранжированном списке, как изменится наша метрика. 3) Поверх результатов КНН с разными параметрами обучаем еще и бустинг из деревьев. Как правило на табличных данных бустинг сильно бьет другие алгоритмы (кроме возможно случайных лесов). Таким образом, мы учли сразу несколько соседей как будто бы. В случае SVD мы не пользуемся дополнительными признаками, а пытаемся предсказывать рейтинг как было в прошлый раз просто через разложение, в такой постановке алгоритм очевидно будет слабее чем lambdaMart в связке с KNN. 